<a href="https://colab.research.google.com/github/alexkychen/FASTQ_tools/blob/master/Genetic_prediction_using_deep_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from google.colab import files

In [0]:
#use a callback to cancel training process if accuracy is over a certain percentage
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.6):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [0]:
myfile = files.upload()

Saving simGenepop.txt to simGenepop.txt


In [0]:
#myfile is a dictionary with filename as key and file content as values

for i in myfile.keys():
  content = myfile.get(i).decode("utf-8")

#separate content into list by line 
content = content.splitlines()
#get first line
first_line = content[0]
print("Imported file description: " + str(first_line))

#check if third item is "pop" and get locus name 
if content[2].strip().lower() == "pop": #for format A
  loci = content[1].strip()
  #split locus name by comma
  loci = loci.split(",")
  loci = [i.strip() for i in loci]
  
else: #for format B
  #searching first "pop"
  i = 0
  for p in content:
    i = i + 1
    if p.strip().lower() == "pop":
      break
  #get locus name
  loci = content[1:i-1]

print("Number of loci: " + str(len(loci)))

#get population and sample size
noPop = 0
ind = 0
popSize = tuple()

for p in content:
  ind = ind + 1
  if p.strip().lower() == "pop":
    if noPop > 0:
      popSize = popSize + (ind-1,)
    noPop = noPop + 1
    ind = 0
#add last pop size to popSize
popSize = popSize + (ind,)

print("Number of populations: "+ str(noPop))
print("Population sizes are: "+str(popSize))
print("Total sample size: " +str(sum(popSize)))

Imported file description: Title line:"A series of simulated samples"
Number of loci: 1000
Number of populations: 3
Population sizes are: (30, 30, 30)
Total sample size: 90


In [0]:
#get sample ID and genetic data
sampleID = []
sampleLoc = []
indata = False

#go through each line
for p in content:
  if p.strip().lower() == "pop":
    indata = True
    continue
  #acquire sample ID and loci from each individual
  if indata:
    onerow = [i.strip() for i in p.split(',')]
    id = onerow[0]
    loc = onerow[1]
    sampleID.append(id)
    sampleLoc.append(loc)

#turn sampleLoc to a numpy array     
sampleLoc = np.asarray([tuple(i.split()) for i in sampleLoc])
print("array size:" + str(sampleLoc.shape))
#print(sampleLoc)

array size:(90, 1000)


In [0]:
#get split size from a locus
splitSize = int(len(sampleLoc[0][0])/2)

#split each locus
locSplit = np.split(sampleLoc, len(loci), axis=1)

#split alleles
sampleAle = np.array(locSplit).view((str,splitSize))
print("(loci, individuals, ploidy) = " + str(sampleAle.shape))

#empty array for genetic data
geneArray = np.array([]).reshape(len(sampleID),0) 

#get unique allele of each locus
for i in range(len(loci)):
  alleles = np.unique(sampleAle[i])
  #print(alleles)
  #one hot encode alleles
  #for diploid
  firstAleIndex = np.array([np.where(alleles==j)[0][0] for j in sampleAle[i][:,0]])
  init = np.zeros((len(sampleID), len(alleles)))
  init[np.arange(len(sampleID)), firstAleIndex] = 0.5
  
  secndAleIndex = np.array([np.where(alleles==k)[0][0] for k in sampleAle[i][:,1]])
  init2 = np.zeros((len(sampleID), len(alleles)))
  init2[np.arange(len(sampleID)), secndAleIndex] = 0.5
  
  #half_one_hot encode for a locus
  init = init + init2
  
  #remove missing value ("00" or "000") columns
  if int(alleles[0]) == 0:
    init = init[:,1:]
  
  #concatenate arrays 
  if init.shape[1] != 0:
    geneArray = np.concatenate((geneArray, init), axis=1)
  #for haploid 
  # (reserved) 
  
print("Shape of genetic data (individuals, alleles) = " + str(geneArray.shape))

(loci, individuals, ploidy) = (1000, 90, 2)
Shape of genetic data (individuals, alleles) = (90, 2000)


In [0]:
#create label array
popArray = np.zeros((len(sampleID), noPop))
#for loop each pop
start = 0
for p in range(noPop):
  end = start + popSize[p]
  popArray[start:end, p] = 1
  start = end

#popArray

In [0]:
#shuffle individuals
s = np.arange(len(sampleID))
np.random.shuffle(s)

geneArrayS = geneArray[s]
popArrayS = popArray[s]

trainProp = 0.7
lastTrainInd = int(len(sampleID)*trainProp)
train_data, test_data = geneArrayS[0:lastTrainInd+1], geneArrayS[lastTrainInd+1:]
train_label, test_label = popArrayS[0:lastTrainInd+1], popArrayS[lastTrainInd+1:]

In [0]:
#define model
model = tf.keras.Sequential([tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(3, activation=tf.nn.softmax)])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_label, epochs=10)

Epoch 1/10
63/63 [==============================] - 0s 4ms/sample - loss: 12.7461 - acc: 0.3175
Epoch 2/10
63/63 [==============================] - 0s 1ms/sample - loss: 14.0784 - acc: 0.3333
Epoch 3/10
63/63 [==============================] - 0s 1ms/sample - loss: 1.2928 - acc: 0.4286
Epoch 4/10
63/63 [==============================] - 0s 1ms/sample - loss: 0.9568 - acc: 0.5714
Epoch 5/10
63/63 [==============================] - 0s 1ms/sample - loss: 1.0814 - acc: 0.5238
Epoch 6/10
63/63 [==============================] - 0s 1ms/sample - loss: 0.8522 - acc: 0.5397
Epoch 7/10
63/63 [==============================] - 0s 1ms/sample - loss: 1.3105 - acc: 0.4286
Epoch 8/10
63/63 [==============================] - 0s 1ms/sample - loss: 0.7621 - acc: 0.5714
Epoch 9/10
63/63 [==============================] - 0s 1ms/sample - loss: 1.0821 - acc: 0.5714
Epoch 10/10
63/63 [==============================] - 0s 1ms/sample - loss: 0.6462 - acc: 0.6984


In [0]:
#evaluate on test data
results = model.evaluate(test_data, test_label)
print(results)

27/27 [==============================] - 0s 5ms/sample - loss: 0.9981 - acc: 0.4815
[0.9981279969215393, 0.4814815]


(7, 3)

In [0]:
input_x = np.random.random(size=(30, 100)) #30 indiviauls and 100 loci
input_y = np.random.randint(2, size=(30,1)) #one label (pop) for 30 individuals

In [0]:
pop_A = np.random.randint(1,10, size=(30,100)) /24
pop_B = np.random.randint(8,17, size=(30,100)) / 24
pop_C = np.random.randint(15,24, size=(30,100)) / 24

array_0 = np.zeros((30,1))
array_1 = np.ones((30,1))

pop_A_label = np.concatenate((array_1, array_0, array_0), axis=1)
pop_B_label = np.concatenate((array_0, array_1, array_0), axis=1)
pop_C_label = np.concatenate((array_0, array_0, array_1), axis=1)

train_data = np.concatenate((pop_A,pop_B,pop_C), axis=0) #shape = (90, 100) 90 individuals and 100 loci
train_label = np.concatenate((pop_A_label,pop_B_label,pop_C_label), axis=0)

print("train data shape: " + str(train_data.shape))
print("train label shape: "+ str(train_label.shape))


train data shape: (90, 100)
train label shape: (90, 3)


In [0]:
train_data[0:4, 0:10]

array([[0.125     , 0.125     , 0.33333333, 0.04166667, 0.375     ,
        0.33333333, 0.20833333, 0.04166667, 0.04166667, 0.29166667],
       [0.08333333, 0.16666667, 0.29166667, 0.125     , 0.20833333,
        0.125     , 0.16666667, 0.125     , 0.04166667, 0.04166667],
       [0.25      , 0.375     , 0.375     , 0.375     , 0.08333333,
        0.29166667, 0.33333333, 0.29166667, 0.25      , 0.375     ],
       [0.08333333, 0.375     , 0.29166667, 0.125     , 0.33333333,
        0.08333333, 0.33333333, 0.375     , 0.25      , 0.25      ]])

In [0]:
#shuffle the data
s = np.arange(train_data.shape[0])
np.random.shuffle(s)
s

array([ 3, 73,  7, 43, 18, 35, 86, 42, 40,  9, 71, 64, 74, 75, 37, 31, 72,
       47, 10, 30,  4, 20, 77, 26, 53, 79,  1, 22, 60, 16, 33, 38, 21, 65,
       87, 81, 13, 63, 80, 29, 52, 34, 56, 41, 57, 27, 61, 55, 50, 66,  6,
       12, 23, 83, 67, 69, 28, 85, 15, 82, 89, 49,  0, 39, 76, 11, 58, 68,
       88, 25, 54, 46, 36, 45, 44, 24, 59, 19, 62, 48,  2, 84,  5, 32, 78,
        8, 17, 51, 14, 70])

In [0]:
train_data = train_data[s]
train_label = train_label[s]

In [0]:
#define model
model = tf.keras.Sequential([tf.keras.layers.Dense(128, activation=tf.nn.relu),
                             #tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(128, activation=tf.nn.relu),
                             #tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(3, activation=tf.nn.softmax)])

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#model.fit(input_x, input_y, epochs=10, callbacks=[callbacks])
history = model.fit(train_data, train_label, epochs=10)

Epoch 1/10
90/90 [==============================] - 0s 977us/sample - loss: 1.2075 - acc: 0.6556
Epoch 2/10
90/90 [==============================] - 0s 109us/sample - loss: 0.4588 - acc: 0.9667
Epoch 3/10
90/90 [==============================] - 0s 107us/sample - loss: 0.4319 - acc: 0.8333
Epoch 4/10
90/90 [==============================] - 0s 99us/sample - loss: 0.3991 - acc: 0.8889
Epoch 5/10
90/90 [==============================] - 0s 101us/sample - loss: 0.4067 - acc: 0.8333
Epoch 6/10
90/90 [==============================] - 0s 109us/sample - loss: 0.3457 - acc: 0.9444
Epoch 7/10
90/90 [==============================] - 0s 86us/sample - loss: 0.3532 - acc: 0.9333
Epoch 8/10
90/90 [==============================] - 0s 110us/sample - loss: 0.3276 - acc: 0.9778
Epoch 9/10
90/90 [==============================] - 0s 122us/sample - loss: 0.4729 - acc: 0.6778
Epoch 10/10
90/90 [==============================] - 0s 97us/sample - loss: 0.3416 - acc: 0.8889
{'loss': [1.2074890481101141, 0.

In [0]:
test_data = np.random.randint(1,10, size=(5,100)) /24

array_0_test = np.zeros((5,1))
array_1_test = np.ones((5,1))

test_label = np.concatenate((array_1_test, array_0_test, array_0_test), axis=1)
results = model.evaluate(test_data, test_label)
print(results)

5/5 [==============================] - 0s 299us/sample - loss: 0.1838 - acc: 1.0000
[0.18382574617862701, 1.0]


In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  6464      
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  4160      
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  195       
Total params: 10,819
Trainable params: 10,819
Non-trainable params: 0
_________________________________________________________________
